## CNN

In [1]:
# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [2]:
from torch import Tensor
import torch

import typing
from typing import List, Tuple

In [5]:
import lincoln as lnc
from lincoln.layers import Dense, Conv2D, Layer
from lincoln.losses import LogLoss, LogSigmoidLoss, MeanSquaredError
from lincoln.optimizers import SGD
from lincoln.operations import Sigmoid, LogSigmoid, ReLU
from lincoln.network import NeuralNetwork

In [6]:
from torchvision.datasets import MNIST
mnist_trainset = MNIST(root="./explanatory/data", train=True, download=False, transform=None)
mnist_testset = MNIST(root="./explanatory/data", train=False, download=True, transform=None)

#### One hot encode labels

In [7]:
data = mnist_trainset
num_labels = len(data.train_labels)
train_labels = torch.zeros(num_labels, 10)
for i in range(num_labels):
    train_labels[i][data.train_labels[i]] = 1
train_labels

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  1.,  0.]])

In [8]:
data = mnist_testset
num_labels = len(data.test_labels)
test_labels = torch.zeros(num_labels, 10)
for i in range(num_labels):
    test_labels[i][data.test_labels[i]] = 1
test_labels

tensor([[ 0.,  0.,  0.,  ...,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  ...,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]])

In [9]:
mnist_train = mnist_trainset.train_data.type(torch.float32).unsqueeze(3) / 255.0
mnist_test = mnist_testset.test_data.type(torch.float32).unsqueeze(3) / 255.0

In [ ]:
%%timeit -n 1 -r 1
out_2 = conv_nn.forward(mnist_train[:2])

In [ ]:
out_2 = conv_nn.forward(mnist_train[:2])

In [ ]:
out_2.shape

In [12]:
X_train = mnist_train
X_test = mnist_test
y_train = train_labels
y_test = test_labels

### Sigmoid + MSE

In [13]:
conv_nn = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=10,
                   activation=ReLU(),
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
            optimizer = SGD())

In [14]:
conv_nn.fit(X_train, y_train, X_test, y_test,  
        epochs = 1,
        eval_every = 1,
        batch_size=1,
        seed=91418);

AssertionError: 
    Two tensors should have the same shape; instead, first Tensor's shape is (10, 10)
    and second Tensor's shape is (1, 10).
    

### LogSigmoid + LogSigmoidLoss

In [15]:
conv_nn_2 = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=10,
                   activation=ReLU(),
                   flatten=True),
            Dense(neurons=10, 
                  activation=LogSigmoid())],
            loss = LogSigmoidLoss(),
            optimizer = SGD())

In [16]:
conv_nn_2.fit(X_train, y_train, X_test, y_test,  
        epochs = 1,
        eval_every = 1,
        batch_size=1,
        seed=91418);

AssertionError: 
    Two tensors should have the same shape; instead, first Tensor's shape is (10, 1)
    and second Tensor's shape is (1, 10).
    

### Sigmoid + LogLoss

In [17]:
conv_nn_3 = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=10,
                   activation=ReLU(),
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogLoss(),
            optimizer = SGD())

In [18]:
conv_nn_3.fit(X_train, y_train, X_test, y_test,  
        epochs = 1,
        eval_every = 1,
        batch_size=1,
        seed=91418);

AssertionError: 
    Two tensors should have the same shape; instead, first Tensor's shape is (10, 1)
    and second Tensor's shape is (1, 10).
    